In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


import getpass  # to get the password without showing the input
password = getpass.getpass()

In [ ]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

NO ME FUNCIONA LA CONEXIÓN

In [ ]:
query: SELECT
    f.film_id,
    f.title,
    f.release_year,
    f.rental_duration,
    f.rental_rate,
    f.length,
    f.replacement_cost,
    f.rating,
    COUNT(r.rental_id) AS total_rentals
FROM
    film f
LEFT JOIN
    inventory i ON f.film_id = i.film_id
LEFT JOIN
    rental r ON i.inventory_id = r.inventory_id
GROUP BY
    f.film_id
ORDER BY
    total_rentals DESC;
    
data = pd.read_sql_query(query, engine)
data

In [ ]:
data.shape, data.dtypes

In [ ]:
data.isna().sum()

In [ ]:
data.describe()

In [ ]:
categorical_cols = ['rating'] 
numerical_cols = ['rental_duration', 'rental_rate', 'replacement_cost']

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ])

In [ ]:
transformed_data = preprocessor.fit_transform(df)

In [ ]:
transformed_df = pd.DataFrame(transformed_data, columns=categorical_cols + numerical_cols)

In [ ]:
print(transformed_df.head())

In [ ]:
from datetime import datetime, timedelta

# Calculate the first day of the previous month
previous_month_start = datetime.today().replace(day=1) - timedelta(days=1)
previous_month_start = previous_month_start.replace(day=1)

# Calculate the last day of the previous month
previous_month_end = previous_month_start.replace(day=1) - timedelta(days=1)

# Filter rentals for the previous month
rentals_last_month = df[(df['rental_date'] >= previous_month_start) & (df['rental_date'] <= previous_month_end)]

# Create a set of film_ids that were rented last month
rented_last_month_set = set(rentals_last_month['film_id'])

# Create a new column indicating whether the film was rented last month or not
df['rented_last_month'] = df['film_id'].apply(lambda x: x in rented_last_month_set)


print(df.head())

In [ ]:
eatures = transformed_df.drop('rented_last_month', axis=1)
target = transformed_df['rented_last_month']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


model = LogisticRegression()


model.fit(X_train, y_train)


predictions = model.predict(X_test)


print(classification_report(y_test, predictions))